In [1]:
import re
import os
from langdetect import detect

def extract_paragraphs(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    # Разбиваем текст на абзацы (по двойному переводу строки)
    paragraphs = re.split(r'\n{2,}', text)

    # Фильтруем абзацы: начинаются с большой буквы и содержат ≥ 500 символов
    filtered_paragraphs = []
    for paragraph in paragraphs:
        if len(paragraph.strip()) == 0:
            continue
        
        if paragraph[0].isupper() and len(re.findall(r'\b\w+\b', paragraph)) >= 100:
            if detect(paragraph) != 'ru': continue
            par = paragraph[:500].replace('\n', ' ')
            for i in range(500, 540):
                par += paragraph[i]
                if paragraph[i] in (' ', '\n', '\t'):break
            filtered_paragraphs.append(par)

    return "\n\n".join(filtered_paragraphs)

cleaned_text = extract_paragraphs("data/human_texts/3 Биология/4.txt")
with open("data/filtered_human.txt", "w", encoding="utf-8") as file:
    file.write(cleaned_text)

path = 'data/human_texts'
with open("data/filtered_human.txt", "w", encoding="utf-8") as f:
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                cleaned_text = extract_paragraphs(file_path)
                print(cleaned_text)
                
                f.write(cleaned_text)
            except Exception: print('error ')

В статье обоснованы методологические основы военно-экономической эффективности ин- тегрированной системы  материально-технического обеспечения военной организации го- сударства, приведены модели и схемы  научно-методологического аппарата ее оценки. Ключевые слова: военно-экономическая эффективность, интегрированная система,  мате- риально-техническое обеспечение, военная организация государства, материальные сред- ства, научно-методологический аппарат, программно-целевой подход.   The article substantiates 

Военная наука, в том числе и теория тыла ветствии с проводимой Правительством РФ поли- (материально-технического обеспечения (МТО)) тикой ориентации на повышение эффективности  как одна из ее отраслей, основываются на базе интегрированной системы МТО (ИСМТО) воен- достижений широкого круга общественно-по- ной организации государства, разработка методо- литических, экономических, военно-теорети- логических основ ее оценки является важнейшим  ческих знаний и конкретной практики военн

KeyboardInterrupt: 

In [3]:
from sklearn.model_selection import train_test_split

with open('data/filtered_human.txt') as f:
    human_all = [l for l in f.readlines() if l!='\n' and len(l) < 540]
    list_human_texts, human_texts_for_lr = train_test_split(human_all, test_size=0.4, random_state=42)
    print(len(list_human_texts))
    print(len(human_texts_for_lr))
    for i in range(4):
        print(len(list_human_texts[i]))
    sample_h = list_human_texts[42]
    print(sample_h)

321
215
503
508
513
505
Взаимодействуя со своим ближним и дальним окружением, используя его ресурсный  потенциал, метрополия испытывает многоаспектную пространственную трансформацию. Она  создают и развивает сферу своего влияния и одновременно видоизменяет свой пространственный  облик, «вбирая» в себя прилегающие к нему сельские поселения и мелкие «пригородские»  посёлки, занимая всё новые и новые территории под селитебную, производственную и иную   застройку. При этом для неё вполне характерна общая, уже достаточно давно 



In [ ]:
with open('data/ai_texts.txt') as f:
    list_ai_texts = [l for l in f.readlines() if l != '\n']
    print(len(list_ai_texts))
    for i in range(4):
        print(len(list_ai_texts[i])) 
    sample_ai = list_ai_texts[42]
    print(sample_ai)

2648
630
570
501
518
499
Робототехника в промышленности — это автоматизированные линии и коллаборативные роботы (cobots), повышающие эффективность. В быту роботы-пылесосы, газонокосилки и ассистенты упрощают жизнь. Развивается сервисная робототехника для логистики и ухода. Ключевые тренды — AI-интеграция, повышенная автономность и тактильная чувствительность. Роботы становятся неотъемлемой частью повседневности, меняя образ жизни и работы.



In [ ]:
# import sys
# print(sys.executable)
# !pip list | grep langdetect

/Library/Developer/CommandLineTools/usr/bin/python3
langdetect                1.0.9


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import OneClassSVM
import nltk
from nltk.corpus import stopwords
import numpy as np
import spacy
import string
import nltk
from nltk.corpus import stopwords
from textstat import flesch_kincaid_grade, flesch_reading_ease
import mlflow.sklearn
from datetime import datetime

nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")
nltk.download('stopwords')
stop_words = set(stopwords.words('russian')).union(set(stopwords.words('english'))) 

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
stop_words = set(stopwords.words('russian')).union(set(stopwords.words('english'))) 
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words=list(stop_words)
)

X_human = vectorizer.fit_transform(human_texts_for_lr)
model_one_class = OneClassSVM(kernel="linear")
zero_labels = np.zeros(len(human_texts_for_lr))
model_one_class.fit(X_human)

mlflow.set_experiment(f'{datetime.now()}  experiment')
for model in [vectorizer, model_one_class]:
    with mlflow.start_run() as run:
        mlflow.sklearn.log_model(model, type(model).__name__.lower())
        run_id = run.info.run_id
        print(f"RUN_ID для загрузки: {run_id}")
        print(type(model).__name__.lower())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/iaroslav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025/11/27 22:39:50 INFO mlflow.tracking.fluent: Experiment with name '2025-11-27 22:39:50.873441  experiment' does not exist. Creating a new experiment.
2025/11/27 22:39:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 22:39:50 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/11/27 22:39:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/27 22:39:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


RUN_ID для загрузки: a84b485bce4d4ff49d13b5f43ebf2244
tfidfvectorizer


2025/11/27 22:39:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RUN_ID для загрузки: ad3acc17f8a244c9873a21a923c6424f
oneclasssvm


In [ ]:
from process_texts import ExtractComponents

ec_human = ExtractComponents(sample_h,vectorizer, model_one_class,
                                      nlp_ru, nlp_en, stop_words)
ec_ai = ExtractComponents(sample_ai,vectorizer, model_one_class,
                                      nlp_ru, nlp_en, stop_words)  

In [ ]:
print(ec_human.text[:-1])
print(ec_human.sentences[:-1])
print(ec_human.words, end='\n\n')

features = ec_human.zz_get_all_features()
print(features)


При всём этом необходимо отметить тот факт, что несмотря на смену этапа  автоматизации в эволюции развития систем управления и комплексов связи уже давно пришли  этапы интеграции и унификации, а в последние годы наметился и переход к этапу  интеллектуализации [2], в развитии технологий и средств контроля наблюдается некоторое  отставание. Лишь недавно появились автоматизированные измерительные комплексы (АИК),  выполняющие узконаправленные функции в интересах конкретного вида ВС РФ, министерства, 
При всём этом необходимо отметить тот факт, что несмотря на смену этапа  автоматизации в эволюции развития систем управления и комплексов связи уже давно пришли  этапы интеграции и унификации, а в последние годы наметился и переход к этапу  интеллектуализации [2], в развитии технологий и средств контроля наблюдается некоторое  отставание. Лишь недавно появились автоматизированные измерительные комплексы (АИК),  выполняющие узконаправленные функции в интересах конкретного вида ВС РФ, министерс

In [ ]:
print(ec_ai.text[:-1])
print(ec_ai.sentences[:-1])
print(ec_ai.words, end='\n\n')

methods = [method for method in dir(ec_ai) if not method.startswith('_') and callable(getattr(ec_ai, method))][:-1]

for method_name in methods:
    try:
        method = getattr(ec_ai, method_name)
        result = method()
        print(f"{method_name}: {result}")
    except Exception as e:
        print(f"{method_name}: ERROR - {e}")

При всём этом необходимо отметить тот факт, что несмотря на смену этапа  автоматизации в эволюции развития систем управления и комплексов связи уже давно пришли  этапы интеграции и унификации, а в последние годы наметился и переход к этапу  интеллектуализации [2], в развитии технологий и средств контроля наблюдается некоторое  отставание. Лишь недавно появились автоматизированные измерительные комплексы (АИК),  выполняющие узконаправленные функции в интересах конкретного вида ВС РФ, министерства, 
Робототехника в промышленности — это автоматизированные линии и коллаборативные роботы (cobots), повышающие эффективность. В быту роботы-пылесосы, газонокосилки и ассистенты упрощают жизнь. Развивается сервисная робототехника для логистики и ухода. Ключевые тренды — AI-интеграция, повышенная автономность и тактильная чувствительность. Роботы становятся неотъемлемой частью повседневности, меняя образ жизни и работы.
['Робототехника в промышленности — это автоматизированные линии и коллаборатив

In [ ]:
from typing import Literal
def final_extract_components(list_texts: list, data_type:Literal['human', 'ai']):
    array_features = []
    k = 1
    for txt in list_texts:
        k += 1
        X = ExtractComponents(txt,vectorizer, model_one_class,
                                      nlp_ru, nlp_en, stop_words)
        features = X.zz_get_all_features()
        array_features.append(features)
        if k % 100 == 0:
            print(len(array_features))
    print(f"Сохранено {len(array_features)} векторов признаков")
    try:
        np.save(f'data/{data_type}_features.npy', array_features)
    except Exception: #in case something went wrong because the script is running quite long
        return array_features

In [79]:
array_features = final_extract_components(list_ai_texts, 'ai')

/var/folders/sl/c1q8vzl53xl0g5kgw_r8t_bh0000gn/T/ipykernel_5850/1189490989.py:99: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(prediction)


49
99
149
199
249
299
349
399
449
499
549
599
649
699
749
799
849
899
949
999
1049
1099
1149
1199
1249
1299
1349
1399
1449
1499
1549
1599
1649
1699
1749
1799
1849
1899
1949
1999
2049
2099
2149
2199
2249
2299
2349
2399
2449
2499
2549
2599
Сохранено 2648 векторов признаков


In [62]:
array_features = final_extract_components(list_human_texts, 'human')

27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
27
Сохранено 6847 векторов признаков


In [ ]:
new_array = []
for lst in array_features:
    new_lst = []
    for x in lst:
        new_lst.append(float(x))
    new_array.append(new_lst)

if new_array[0]:
    np.save('data/human_features.npy', np.array(new_array))
    len(new_array)

/var/folders/sl/c1q8vzl53xl0g5kgw_r8t_bh0000gn/T/ipykernel_5850/2803850709.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  new_lst.append(float(x))


[2.9577464788732395,
 0.0,
 8.0,
 11.0,
 4.0,
 0.0,
 2.0,
 2.0,
 0.0,
 22.0,
 0.0,
 2.0,
 4.0,
 0.0,
 4.0,
 3.0,
 0.0,
 7.0,
 0.0,
 16.4,
 5.0,
 104.72625000000002,
 2.9375,
 13.8,
 1.0,
 6.130434782608695,
 0.98]